In [17]:
from pycocotools.coco import COCO
import cv2
import numpy as np
import json

def getSupercategory(category_id):
    f = open(coco_annotation_file_path)
    cat_json_file = json.load(f)

    for category in cat_json_file['categories']:
        if category_id == category['id']:
            f = open('data/supercategories.json')
            supcat_json_file = json.load(f)
            return supcat_json_file[category['supercategory']]
            
coco_annotation_file_path = 'data/annotations_0_val.json'
coco_annotation = COCO(annotation_file=coco_annotation_file_path)      
img_ids = coco_annotation.getImgIds()

for index in range(len(img_ids)):
    print(index, end='\r')
    img_id = img_ids[index]
    img_info = coco_annotation.loadImgs(img_id)[0]

    img_path = 'data/' + img_info["file_name"]
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (512, 512), interpolation = cv2.INTER_NEAREST)
    
    
    # Get masks
    ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
    anns = coco_annotation.loadAnns(ann_ids)
    mask = np.zeros((img_info['height'],img_info['width']))

    # Use supercategories instead classes
    for ann in anns:
        mask = np.maximum(mask, coco_annotation.annToMask(ann) * getSupercategory(ann['category_id']))
    
    mask = cv2.resize(mask, (512, 512), interpolation = cv2.INTER_NEAREST)
    
    mask_filename = img_info["file_name"].replace('images/','').replace('.jpg', '.png')
    path_to_image = 'data/images/resized/' + img_info["file_name"]
    path_to_mask = 'data/images/resized/masks/' + mask_filename

    cv2.imwrite(path_to_image, image)
    cv2.imwrite(path_to_mask, mask)
    cv2.waitKey(0)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
